<a href="https://colab.research.google.com/github/aj1365/RWVUNet/blob/main/ResidualwavevisionU_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import cv2 # For CV operations
from PIL import Image  #To create and store images
import numpy as np
import os
from patchify import patchify
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow.keras.backend as K
from typing import Callable

In [ ]:
images = []
originalImages = os.listdir("E:/Flood mapping/VH/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/Flood mapping/VH/" + str(image))
    img = img.resize((256, 256))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

VH=images
VH = np.array(VH)
VH.shape


images = []
originalImages = os.listdir("E:/Flood mapping/labels/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/Flood mapping/labels/" + str(image))
    img = img.resize((256, 256))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

labels=images
labels = np.array(labels)

labels.shape


images = []
originalImages = os.listdir("E:/Flood mapping/VV/")

for index,image in enumerate(originalImages):
    print("Image number : " +str(index) )
    img = Image.open("E:/Roy/Flood mapping/VV/" + str(image))
    img = img.resize((256, 256))
    arr = np.array(img)
    #arr = np.expand_dims(arr, -1)
    images.append(arr)

VV=images
VV = np.array(VV)
VV.shape

VV=VV.reshape((VV.shape[0],VV.shape[1],VV.shape[2],1))
VH=VH.reshape((VH.shape[0],VH.shape[1],VH.shape[2],1))


S1=VV+VH
S2=VH-VV
S3=VV*VV
S4=VH*VH
S5=VV*VH
S6=(VV+VH)*(VH-VV)

meanVV = np.mean(VV)  # mean for data centering
stdVV = np.std(VV)  # std for data normalization

VV -= meanVV
VV /= stdVV


meanVH = np.mean(VH)  # mean for data centering
stdVH = np.std(VH)  # std for data normalization

VH -= meanVH
VH /= stdVH

meanS1 = np.mean(S1)  # mean for data centering
stdS1 = np.std(S1)  # std for data normalization

S1 -= meanS1
S1 /= stdS1


meanS2 = np.mean(S2)  # mean for data centering
stdS2 = np.std(S2)  # std for data normalization

S2 -= meanS2
S2 /= stdS2


meanS3 = np.mean(S3)  # mean for data centering
stdS3 = np.std(S3)  # std for data normalization

S3 -= meanS3
S3 /= stdS3


meanS4 = np.mean(S4)  # mean for data centering
stdS4 = np.std(S4)  # std for data normalization

S4 -= meanS4
S4 /= stdS4


meanS5 = np.mean(S5)  # mean for data centering
stdS5 = np.std(S5)  # std for data normalization

S5 -= meanS5
S5 /= stdS5


meanS6 = np.mean(S6)  # mean for data centering
stdS6 = np.std(S6)  # std for data normalization

S6 -= meanS6
S6 /= stdS6

SAR=np.concatenate((VV, VH, S1, S2, S3, S4, S5, S6) , axis = 3)


In [ ]:
SAR=np.concatenate((VV, VH, S1, S3, S4, S5, S6) , axis = 3)
labels=labels.reshape((labels.shape[0],labels.shape[1],labels.shape[2],1))

from sklearn.model_selection import train_test_split

TrainX, TestX, TrainY, TestY = train_test_split(SAR,
                                                   labels,
                                                    test_size=0.2,
                                                    random_state=0)

In [ ]:
with h5py.File("E:/Flood mapping/Data/Dataset_train.h5", 'w') as hdf:
    hdf.create_dataset('images', data=TrainX, compression='gzip', compression_opts=9)
    hdf.create_dataset('masks', data=TrainY, compression='gzip', compression_opts=9)

In [ ]:
def DiceLoss(targets, inputs, smooth=1e-6):


    intersection = K.sum(targets *inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

def IoULoss(targets, inputs, smooth=1e-6):



    intersection = K.sum(targets *inputs)
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection

    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU

In [ ]:
# Metrics to be used when evaluating the network

from tensorflow_addons.metrics import F1Score

precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
f1 = F1Score(num_classes=2, name='f1', average='micro', threshold=0.4)
sgd_optimizer = Adam()

In [ ]:
from keras_cv_attention_models import backend
from keras_cv_attention_models.backend import layers, models, functional
#from keras_cv_attention_models.models import register_model
from keras_cv_attention_models.attention_layers import (
    activation_by_name,
    batchnorm_with_activation,
    conv2d_no_bias,
    drop_block,
    group_norm,
    # mlp_block, # cannot import name 'mlp_block' due to circular import
    add_pre_post_process,
)
from keras_cv_attention_models.download_and_load import reload_model_weights


In [ ]:
def mlp_block(inputs, hidden_dim, out_channel, drop_rate=0, activation="gelu"):


    nn = layers.Conv2D(hidden_dim, kernel_size=1, use_bias=True)(inputs)
    nn = activation_by_name(nn, activation)
    nn = layers.Dropout(drop_rate) if drop_rate > 0 else nn
    nn = layers.Conv2D(out_channel, kernel_size=1, use_bias=True)(nn)
    nn = layers.Dropout(drop_rate) if drop_rate > 0 else nn

    return nn

def phase_aware_token_mixing(inputs, out_channel=-1, qkv_bias=False, output_dropout=0, activation="gelu"):

    input_channel = inputs.shape[-1] if backend.image_data_format() == "channels_last" else inputs.shape[1]
    out_channel = out_channel if out_channel > 0 else input_channel

    theta_h = conv2d_no_bias(inputs, out_channel, kernel_size=1, use_bias=True)
    theta_h = batchnorm_with_activation(theta_h, activation="relu")  # Fixed as relu [ ??? ]
    height = conv2d_no_bias(inputs, out_channel, kernel_size=1, use_bias=qkv_bias)
    # height = layers.Concatenate(axis=-1)([height * functional.cos(theta_h), height * functional.sin(theta_h)])
    height_cos = layers.Multiply()([height, functional.cos(theta_h)])
    height_sin = layers.Multiply()([height, functional.sin(theta_h)])
    height = layers.Concatenate(axis=-1 if backend.image_data_format() == "channels_last" else 1)([height_cos, height_sin])
    height = conv2d_no_bias(height, out_channel, kernel_size=(1, 7), padding="same", groups=out_channel, use_bias=False)

    theta_w = conv2d_no_bias(inputs, out_channel, kernel_size=1, use_bias=True)
    theta_w = batchnorm_with_activation(theta_w, activation="relu")  # Fixed as relu [ ??? ]
    width = conv2d_no_bias(inputs, out_channel, kernel_size=1, use_bias=qkv_bias)
    # width = layers.Concatenate(axis=-1)([width * functional.cos(theta_w), width * functional.sin(theta_w)])
    width_cos = layers.Multiply()([width, functional.cos(theta_w)])
    width_sin = layers.Multiply()([width, functional.sin(theta_w)])
    width = layers.Concatenate(axis=-1 if backend.image_data_format() == "channels_last" else 1)([width_cos, width_sin])
    width = conv2d_no_bias(width, out_channel, kernel_size=(7, 1), padding="same", groups=out_channel, use_bias=False)

    channel = conv2d_no_bias(inputs, out_channel, kernel_size=1, use_bias=qkv_bias)

    # print(f"{height.shape = }, {width.shape = }, {channel.shape = }, {out_channel = }")
    nn = layers.Add()([height, width, channel])
    nn = layers.GlobalAveragePooling2D(keepdims=True)(nn)
    nn = mlp_block(nn, out_channel // 4, out_channel=out_channel * 3, activation=activation)
    nn = layers.Reshape([1, 1, out_channel, 3] if backend.image_data_format() == "channels_last" else [out_channel, 1, 1, 3])(nn)
    nn = layers.Softmax(axis=-1)(nn)
    attn_height, attn_width, attn_channel = functional.unstack(nn, axis=-1)
    # attn = layers.Add()([height * attn_height, width * attn_width, channel * attn_channel])
    attn_height = layers.Multiply()([height, attn_height])
    attn_width = layers.Multiply()([width, attn_width])
    attn_channel = layers.Multiply()([channel, attn_channel])
    # print(f"{attn_height.shape = }, {attn_width.shape = }, {attn_channel.shape = }")
    attn = layers.Add()([attn_height, attn_width, attn_channel])

    out = conv2d_no_bias(attn, out_channel, kernel_size=1, use_bias=True)
    out = layers.Dropout(output_dropout)(out) if output_dropout > 0 else out

    return out


def wave_block(inputs, qkv_bias=False, mlp_ratio=4, use_group_norm=False, drop_rate=0, activation="gelu"):

    input_channel = inputs.shape[-1] if backend.image_data_format() == "channels_last" else inputs.shape[1]

    attn = group_norm(inputs, groups=1) if use_group_norm else batchnorm_with_activation(inputs, activation=None)
    attn = phase_aware_token_mixing(attn, out_channel=input_channel, qkv_bias=qkv_bias, activation=activation)
    attn = drop_block(attn, drop_rate=drop_rate)
    attn_out = layers.Add()([inputs, attn])

    mlp = group_norm(attn_out, groups=1) if use_group_norm else batchnorm_with_activation(attn_out, activation=None)
    mlp = mlp_block(mlp, int(input_channel * mlp_ratio), out_channel=input_channel, activation=activation)
    mlp = drop_block(mlp)

    mlp_out = layers.Add()([inputs, attn_out, mlp])

    return mlp_out



In [ ]:
################################## LIBRARIES ##################################

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose, concatenate, Lambda, UpSampling2D
from tensorflow.keras import Model, Input
from contextlib import redirect_stdout
import tensorflow as tf


############################# CONVOLUTIONAL BLOCK #############################

def conv_block(feature_map):

    # Main Path
    conv_1 = Conv2D(filters=64, kernel_size=(1,1), strides=(1,1), padding='same')(feature_map)
    bn = BatchNormalization()(conv_1)
    relu = Activation(activation='relu')(bn)
    conv_2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(relu)

    res_conn = Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding='same')(feature_map)
    res_conn = BatchNormalization()(res_conn)
    addition = Add()([res_conn, conv_2])

    return addition

############################### RESIDUAL BLOCK ################################

def res_block(feature_map, conv_filter, stride):

    bn_1 = BatchNormalization()(feature_map)
    relu_1 = Activation(activation='relu')(bn_1)
    conv_1 = Conv2D(conv_filter, kernel_size=(3,3), strides=stride[0], padding='same')(relu_1)
    bn_2 = BatchNormalization()(conv_1)
    relu_2 = Activation(activation='relu')(bn_2)
    conv_2 = Conv2D(conv_filter, kernel_size=(3,3), strides=stride[1], padding='same')(relu_2)


    res_conn = Conv2D(conv_filter, kernel_size=(1,1), strides=stride[0], padding='same')(feature_map)
    res_conn = BatchNormalization()(res_conn)
    addition = Add()([res_conn, conv_2])

    return addition

################################### ENCODER ###################################

def encoder(feature_map):

    # Initialize the to_decoder connection
    to_decoder = []

    # Block 1 - Convolution Block
    path = conv_block(feature_map)
    to_decoder.append(path)

    # Block 2 - Residual Block 1
    path = res_block(path, 128, [(2, 2), (1, 1)])
    to_decoder.append(path)

    # Block 3 - Residual Block 2
    path = res_block(path, 128, [(2, 2), (1, 1)])
    to_decoder.append(path)

    return to_decoder

################################### DECODER ###################################

def decoder(feature_map, from_encoder):

    qkv_bias=False
    mlp_ratio=2
    use_group_norm=False
    block_drop_rate=0


    # Block 1: Up-sample, Concatenation + Residual Block 1
    wave_block1 = wave_block(from_encoder[2], qkv_bias, mlp_ratio, use_group_norm, block_drop_rate, activation='ReLU')
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(feature_map)
    # main_path = Conv2DTranspose(filters=256, kernel_size=(2,2), strides=(2,2), padding='same')(feature_map)
    main_path = concatenate([main_path, wave_block1], axis=3)
    main_path = res_block(main_path, 256, [(1, 1), (1, 1)])

    # Block 2: Up-sample, Concatenation + Residual Block 2
    wave_block2 = wave_block(from_encoder[1], qkv_bias, mlp_ratio, use_group_norm, block_drop_rate, activation='ReLU')
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=128, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = concatenate([main_path, wave_block2], axis=3)
    main_path = res_block(main_path, 128, [(1, 1), (1, 1)])

    # Block 3: Up-sample, Concatenation + Residual Block 3
    wave_block3 = wave_block(from_encoder[0], qkv_bias, mlp_ratio, use_group_norm, block_drop_rate, activation='ReLU')
    main_path = UpSampling2D(size=(2,2), interpolation='bilinear')(main_path)
    # main_path = Conv2DTranspose(filters=64, kernel_size=(2,2), strides=(2,2), padding='same')(main_path)
    main_path = concatenate([main_path, wave_block3], axis=3)
    main_path = res_block(main_path, 64, [(1, 1), (1, 1)])

    return main_path

################################ RESIDUAL UNET ################################

def WaveResUNet():

    num_blocks=[4]
    out_channels=[128]
    qkv_bias=False
    mlp_ratios=[2]
    use_group_norm=False
    block_drop_rate=0
    drop_connect_rate=0
    use_downsample_norm=True

    # Input
    model_input = Input(shape=(256, 256, 7))

    # Encoder Path
    model_encoder = encoder(model_input)

    # Bottleneck
    X = res_block(model_encoder[2], 32, [(2, 2), (1, 1)])

    total_blocks = sum(num_blocks)
    global_block_id = 0

    for stack_id, (num_block, out_channel, mlp_ratio) in enumerate(zip(num_blocks, out_channels, mlp_ratios)):
        stage_name = "stack{}_".format(stack_id + 1)

        if stack_id > 0:
            X = conv2d_no_bias(X, out_channel, kernel_size=3, strides=2, padding="same", use_bias=True)
            if use_downsample_norm:

                X = group_norm(X, groups=1) if use_group_norm else batchnorm_with_activation(X, activation=None)

        for block_id in range(num_block):

            block_drop_rate = drop_connect_rate * global_block_id / total_blocks
            global_block_id += 1
            X = wave_block(X, qkv_bias, mlp_ratio, use_group_norm, block_drop_rate, activation='ReLU')

    # Decoder Path
    model_decoder = decoder(X, model_encoder)

    # Output
    output_layer = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1), activation='sigmoid', padding='same')(model_decoder)

    model=Model(inputs=model_input, outputs=output_layer)
    model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1])


    return model

model=WaveResUNet()

In [ ]:
print('*'*30)
print('Loading and preprocessing train data...')
print('*'*30)
file = h5py.File('E:/Flood mapping/Data/Dataset_train.h5', 'r')
imgs_train = file.get('images')
imgs_mask_train = file.get('masks')
imgs_train = np.array(imgs_train)
imgs_mask_train = np.array(imgs_mask_train)

print(imgs_train.shape)
print(imgs_mask_train.shape)

imgs_train = imgs_train.astype('float32')

imgs_mask_train = imgs_mask_train.astype('float32')
#imgs_mask_train /= 255  # scale masks to [0, 1]

print('*'*30)
print('Creating and compiling model...')
print('*'*30)
model=ResUNet()

In [ ]:
weight_decay = 0.0001
import tensorflow_addons as tfa
learning_rate=1e-3
optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )


checkpoint_filepath = "E:/Flood mapping/Data/wave_ResUNet.h5"



#with tf.device('/CPU:0'):
history = model.fit(
        x=imgs_train,
        y=imgs_mask_train,
        batch_size=1,
        epochs=40,
        validation_split=0.3
    )


In [ ]:
testImages=TestX

with h5py.File("E:/Flood mapping/Data/Dataset_test.h5", 'w') as hdf:
    hdf.create_dataset('images', data=testImages, compression='gzip', compression_opts=9)
file = h5py.File('E:/Flood mapping/Data/Dataset_test.h5', 'r')
imgs_test = file.get('images')
#imgs_mask_test = file.get('masks')
imgs_test = np.array(imgs_test)
#imgs_mask_test = np.array(imgs_mask_test)
imgs_test = imgs_test.astype('float32')
#imgs_test -= mean
#imgs_test /= std

print('*'*30)
print('Loading saved weights...')
print('*'*30)
model.load_weights('E:/Flood mapping/Data/wave_ResUNet.h5')

print('*'*30)
print('Predicting masks on test data...')
print('*'*30)
imgs_mask_test = model.predict(imgs_test, verbose=1,batch_size=1)
imgs_mask_test=(imgs_mask_test - np.min(imgs_mask_test))/(np.max(imgs_mask_test) - np.min(imgs_mask_test))
acc = model.evaluate(imgs_test, TestY, batch_size=1)